<a href="https://colab.research.google.com/github/Mimohamed1998/power_opt_simulation/blob/main/matrix_based_implementation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Customer Configurations

In [1]:
customer_cap  = 2
l_status = 3

# Create the dummay dataframe

In [2]:
import numpy as np

In [3]:
import pandas as pd
import random

# Constants
NUM_WINGS = 5
MIN_ROOMS_PER_WING = 3
MAX_ROOMS_PER_WING = 5
NORMAL_ROOMS = 4
LUXURY_ROOMS = 1

# Function to generate random luxury status and energy cost
def generate_luxury_status():
    luxury_status = random.choice([1,2,3])
    if luxury_status == 1:
        return luxury_status, random.randint(70, 100)
    elif luxury_status == 2:
        return luxury_status, random.randint(90, 120)
    else:
        return luxury_status, random.randint(110, 130)

# Initialize lists for DataFrame columns
room_numbers = []
per_room_energy_costs = []
wing_numbers = []
luxury_statuses = []
room_capacities = []

# Create DataFrame
for wing in range(NUM_WINGS):
    wing_rooms = random.randint(MIN_ROOMS_PER_WING, MAX_ROOMS_PER_WING)
    for room in range(1, wing_rooms + 1):
        room_number = room
        luxury_status, energy_cost = generate_luxury_status()
        room_capacity = random.choice([2, 3])

        room_numbers.append(room_number)
        per_room_energy_costs.append(energy_cost)
        wing_numbers.append(chr(ord('A') + wing))
        luxury_statuses.append(luxury_status)
        room_capacities.append(room_capacity)

# Create DataFrame
data = {
    'Room_Number': room_numbers,
    'Per_Room_Energy_Cost': per_room_energy_costs,
    'Wing_Number': wing_numbers,
    'Luxury_Status': luxury_statuses,
    'Room_Capacity': room_capacities
}

hotel_df = pd.DataFrame(data)

# Display the DataFrame
print(hotel_df)


    Room_Number  Per_Room_Energy_Cost Wing_Number  Luxury_Status  \
0             1                    91           A              1   
1             2                    78           A              1   
2             3                    95           A              2   
3             4                    93           A              2   
4             5                   122           A              3   
5             1                   104           B              2   
6             2                    96           B              2   
7             3                   111           B              3   
8             4                   101           B              2   
9             1                    70           C              1   
10            2                    94           C              1   
11            3                   110           C              3   
12            4                   122           C              3   
13            5                   110           

## Testing the dataframe

In [4]:
hotel_df[hotel_df['Room_Number']==1]

,Room_Number,Per_Room_Energy_Cost,Wing_Number,Luxury_Status,Room_Capacity
0,1,91,A,1,3
5,1,104,B,2,2
9,1,70,C,1,2
14,1,119,D,3,2
17,1,95,E,2,2


In [5]:
hotel_df[hotel_df['Wing_Number']=='B'].count()

Room_Number             4
Per_Room_Energy_Cost    4
Wing_Number             4
Luxury_Status           4
Room_Capacity           4
dtype: int64

In [6]:
hotel_df[hotel_df['Room_Capacity']==2].count()

Room_Number             12
Per_Room_Energy_Cost    12
Wing_Number             12
Luxury_Status           12
Room_Capacity           12
dtype: int64

## Create the wing datafrma

In [7]:
import pandas as pd
import random

# Constants
NUM_WINGS = 50
MIN_WING_COST = 300
MAX_WING_COST = 350

# Initialize lists for DataFrame columns
wing_numbers = []
wing_costs = []

# Create DataFrame for wing information
for wing in range(NUM_WINGS):
    wing_number = chr(ord('A') + wing)
    wing_cost = random.uniform(MIN_WING_COST, MAX_WING_COST)

    wing_numbers.append(wing_number)
    wing_costs.append(wing_cost)

# Create DataFrame
wing_data = {
    'Wing_Number': wing_numbers,
    'Wing_Cost': wing_costs
}

wing_df = pd.DataFrame(wing_data)

# Display the DataFrame
print(wing_df)


   Wing_Number   Wing_Cost
0            A  301.247059
1            B  309.530826
2            C  309.649488
3            D  338.931034
4            E  317.585668
5            F  303.697805
6            G  307.967839
7            H  311.477410
8            I  320.586442
9            J  320.750889
10           K  349.883367
11           L  342.990514
12           M  317.097351
13           N  320.170373
14           O  339.108721
15           P  310.089600
16           Q  346.748342
17           R  304.440374
18           S  321.335774
19           T  331.261351
20           U  339.804341
21           V  322.762654
22           W  305.027868
23           X  321.358247
24           Y  336.816411
25           Z  334.704379
26           [  320.810968
27           \  310.232987
28           ]  326.121040
29           ^  327.582776
30           _  318.675173
31           `  300.206748
32           a  345.103492
33           b  330.872120
34           c  315.903249
35           d  300.517927
3

# pre process metric

In [8]:
hotel_df['status'] = True

In [9]:
# Find wing with the highest number of rooms
wing_max_rooms = hotel_df.groupby('Wing_Number')['Room_Number'].count().max()
wing_max_rooms

5

In [10]:
# Create DataFrame with rows from 0 to max_rooms_per_wing
rooms_df = pd.DataFrame({'Room_Number': range(wing_max_rooms )})
rooms_df['Room_Number'] = rooms_df['Room_Number']+1

In [11]:
cross_joined_df = pd.merge(wing_df[['Wing_Number']].assign(key=1), rooms_df.assign(key=1), on='key').drop('key', axis=1)

cross_joined_df

,Wing_Number,Room_Number
0,A,1
1,A,2
2,A,3
3,A,4
4,A,5
...,...,...
245,r,1
246,r,2
247,r,3
248,r,4


In [12]:
# Perform left join with cross_joined_df and fill subset with 65,535
hotel_df_processed = pd.merge(cross_joined_df, hotel_df, on=['Wing_Number', 'Room_Number'], how='left')
# hotel_df_processed['Per_Room Energy_Cost'] = hotel_df_processed['Per_Room_Energy_Cost'].fillna(65535)
hotel_df_processed['Luxury_Status'] = hotel_df_processed['Luxury_Status'].fillna(0)
hotel_df_processed['Room_Capacity'] = hotel_df_processed['Room_Capacity'].fillna(0)
hotel_df_processed['status'] = hotel_df_processed['status'].fillna(False)

# hotel_df_processed = hotel_df_processed.sort('Room Number')

# Display the resulting hotel DataFrame
print("\nHotel DataFrame:")
hotel_df_processed


Hotel DataFrame:


,Wing_Number,Room_Number,Per_Room_Energy_Cost,Luxury_Status,Room_Capacity,status
0,A,1,91.0,1.0,3.0,True
1,A,2,78.0,1.0,3.0,True
2,A,3,95.0,2.0,3.0,True
3,A,4,93.0,2.0,3.0,True
4,A,5,122.0,3.0,2.0,True
...,...,...,...,...,...,...
245,r,1,NaN,0.0,0.0,False
246,r,2,NaN,0.0,0.0,False
247,r,3,NaN,0.0,0.0,False
248,r,4,NaN,0.0,0.0,False


In [13]:
# Sort the DataFrame by 'Wing Number' and 'Room Number'
hotel_df_processed = hotel_df_processed.sort_values(by=['Wing_Number', 'Room_Number'])
hotel_df_processed

,Wing_Number,Room_Number,Per_Room_Energy_Cost,Luxury_Status,Room_Capacity,status
0,A,1,91.0,1.0,3.0,True
1,A,2,78.0,1.0,3.0,True
2,A,3,95.0,2.0,3.0,True
3,A,4,93.0,2.0,3.0,True
4,A,5,122.0,3.0,2.0,True
...,...,...,...,...,...,...
245,r,1,NaN,0.0,0.0,False
246,r,2,NaN,0.0,0.0,False
247,r,3,NaN,0.0,0.0,False
248,r,4,NaN,0.0,0.0,False


# Creating the wing cost

# Create the initial metrices

0. Room ID metric

1. capacity metric

2. per room cost metric

3. wing cost

4. luxury status

In [14]:
# Room id metrics

room_id_metric = np.stack(hotel_df_processed.groupby('Wing_Number')['Room_Number'].apply(list))
room_id_metric = np.array(room_id_metric, dtype="float32")
room_id_metric

array([[1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2., 3., 4., 5.],
       [1., 2.

In [15]:
# Room capacity metric

room_capacity_metric = np.stack(hotel_df_processed.groupby('Wing_Number')['Room_Capacity'].apply(list))
room_capacity_metric = np.array(room_capacity_metric, dtype="float32")
room_capacity_metric

array([[3., 3., 3., 3., 2.],
       [2., 2., 3., 2., 0.],
       [2., 2., 2., 2., 2.],
       [2., 3., 3., 0., 0.],
       [2., 2., 3., 3., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0.

In [16]:
# per room cost metric

room_energy_metric = np.stack(hotel_df_processed.groupby('Wing_Number')['Per_Room_Energy_Cost'].apply(list))
room_energy_metric = np.array(room_energy_metric, dtype="float32")
room_energy_metric

array([[ 91.,  78.,  95.,  93., 122.],
       [104.,  96., 111., 101.,  nan],
       [ 70.,  94., 110., 122., 110.],
       [119.,  72.,  99.,  nan,  nan],
       [ 95.,  98.,  74., 127.,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,

In [17]:
wing_df

,Wing_Number,Wing_Cost
0,A,301.247059
1,B,309.530826
2,C,309.649488
3,D,338.931034
4,E,317.585668
5,F,303.697805
6,G,307.967839
7,H,311.477410
8,I,320.586442
9,J,320.750889


In [18]:
# per room cost metric

room_wing_cost_metric = np.stack(wing_df.groupby('Wing_Number')['Wing_Cost'].apply(list))
room_wing_cost_metric = np.array(room_wing_cost_metric , dtype="float32")
room_wing_cost_metric

array([[301.24707],
       [309.53082],
       [309.64948],
       [338.93103],
       [317.58566],
       [303.6978 ],
       [307.96783],
       [311.47742],
       [320.58646],
       [320.7509 ],
       [349.88336],
       [342.9905 ],
       [317.09735],
       [320.17038],
       [339.10873],
       [310.0896 ],
       [346.74835],
       [304.44037],
       [321.3358 ],
       [331.26135],
       [339.80435],
       [322.76266],
       [305.02786],
       [321.35825],
       [336.8164 ],
       [334.70438],
       [320.81097],
       [310.233  ],
       [326.12103],
       [327.58276],
       [318.67517],
       [300.20676],
       [345.1035 ],
       [330.87213],
       [315.90326],
       [300.5179 ],
       [326.6834 ],
       [337.27936],
       [344.63666],
       [326.5354 ],
       [332.83267],
       [336.49365],
       [323.28754],
       [319.69266],
       [346.02237],
       [312.89716],
       [349.28143],
       [339.2158 ],
       [342.3127 ],
       [333.04224]],

In [19]:
# luxury status

# per room cost metric

room_lux_metric = np.stack(hotel_df_processed.groupby('Wing_Number')['Luxury_Status'].apply(list))
room_lux_metric = np.array(room_lux_metric, dtype="float32")
room_lux_metric

array([[1., 1., 2., 2., 3.],
       [2., 2., 3., 2., 0.],
       [1., 1., 3., 3., 3.],
       [3., 1., 1., 0., 0.],
       [2., 2., 1., 3., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0.

## Get the room and wing occupancies

In [20]:
room_capacity_metric.shape[0]

50

In [21]:
import numpy as np

# Create room_occupancy matrix
room_occupancy = np.zeros(room_capacity_metric.shape, dtype=int)

# Set the first row to be full of ones
room_occupancy[0, :] = 1

# Set the second to third rows with a mix of 1s and 0s
room_occupancy[1:room_capacity_metric.shape[0]-3, :] = np.random.choice([0, 1], size=(46, room_capacity_metric.shape[1]))

# Set the last row to be full of zeros
room_occupancy[room_capacity_metric.shape[0]-1, :] = 0

# Create wing_occupancy matrix as binary
wing_occupancy_binary = np.any(room_occupancy, axis=1).astype(int)

# Display the matrices
print("Room Occupancy Matrix:")
print(room_occupancy)

print("\nWing Occupancy Matrix:")
print(wing_occupancy_binary)


Room Occupancy Matrix:
[[1 1 1 1 1]
 [1 0 1 0 0]
 [1 0 0 1 1]
 [1 1 1 1 0]
 [0 0 1 0 1]
 [0 0 0 0 0]
 [1 0 0 0 1]
 [0 0 0 1 1]
 [1 0 1 1 0]
 [1 1 0 0 1]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 1]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 1 1 1 1]
 [0 0 0 0 0]
 [1 0 0 1 1]
 [0 0 0 1 1]
 [0 1 0 0 0]
 [0 0 0 1 1]
 [0 1 1 1 0]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [1 0 1 1 1]
 [1 0 0 1 0]
 [0 0 1 1 1]
 [1 0 0 1 1]
 [1 1 0 0 1]
 [0 1 1 1 0]
 [0 1 1 1 1]
 [0 0 1 1 0]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 1 1 0 0]
 [0 1 0 1 0]
 [0 0 1 0 0]
 [1 0 0 1 1]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 1 0 1 1]
 [1 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Wing Occupancy Matrix:
[1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 0]


## Processing the metrices

In [22]:
room_occupancy[:7,]

array([[1, 1, 1, 1, 1],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 1],
       [1, 1, 1, 1, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1]])

In [23]:
room_occupancy.shape

(50, 5)

In [24]:
customer_cap
l_status
customer_cap

2

### making binary metrices

In [25]:
room_capacity_metric

array([[3., 3., 3., 3., 2.],
       [2., 2., 3., 2., 0.],
       [2., 2., 2., 2., 2.],
       [2., 3., 3., 0., 0.],
       [2., 2., 3., 3., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0.

In [26]:
# Capacity processing
print('initlal room_capacity_metric')
print(room_capacity_metric)
room_capacity_metric_pr = np.where(room_capacity_metric >= customer_cap, 1, 0)
print('processed room_capacity_metric')
print(room_capacity_metric_pr)

initlal room_capacity_metric
[[3. 3. 3. 3. 2.]
 [2. 2. 3. 2. 0.]
 [2. 2. 2. 2. 2.]
 [2. 3. 3. 0. 0.]
 [2. 2. 3. 3. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
processed room_capacity_metric
[[1 1 1 1 1]
 [1 1 1 1 0]
 [1 1 1 1 1]


In [27]:
# Luxury processing
print('initlal room_lux_metric')
print(room_lux_metric)
room_lux_metric_pr = np.where(room_lux_metric >= l_status, 1, 0)
print('processed room_capacity_metric')
print(room_lux_metric_pr)

initlal room_lux_metric
[[1. 1. 2. 2. 3.]
 [2. 2. 3. 2. 0.]
 [1. 1. 3. 3. 3.]
 [3. 1. 1. 0. 0.]
 [2. 2. 1. 3. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
processed room_capacity_metric
[[0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 1 1 1]
 [1 0

### Inverting metrics

In [28]:
# Room occupancy

print('before_inverting')
print(room_occupancy)
room_occupancy_inv = ~room_occupancy.astype(bool)
room_occupancy_inv = room_occupancy_inv.astype(int)
print('after_inverting')
print(room_occupancy_inv)

before_inverting
[[1 1 1 1 1]
 [1 0 1 0 0]
 [1 0 0 1 1]
 [1 1 1 1 0]
 [0 0 1 0 1]
 [0 0 0 0 0]
 [1 0 0 0 1]
 [0 0 0 1 1]
 [1 0 1 1 0]
 [1 1 0 0 1]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 0 1 0]
 [0 1 0 0 1]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [0 1 1 1 1]
 [0 0 0 0 0]
 [1 0 0 1 1]
 [0 0 0 1 1]
 [0 1 0 0 0]
 [0 0 0 1 1]
 [0 1 1 1 0]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [1 0 1 1 1]
 [1 0 0 1 0]
 [0 0 1 1 1]
 [1 0 0 1 1]
 [1 1 0 0 1]
 [0 1 1 1 0]
 [0 1 1 1 1]
 [0 0 1 1 0]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [1 1 1 0 0]
 [0 1 0 1 0]
 [0 0 1 0 0]
 [1 0 0 1 1]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 1 0 1 1]
 [1 0 0 0 1]
 [1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
after_inverting
[[0 0 0 0 0]
 [0 1 0 1 1]
 [0 1 1 0 0]
 [0 0 0 0 1]
 [1 1 0 1 0]
 [1 1 1 1 1]
 [0 1 1 1 0]
 [1 1 1 0 0]
 [0 1 0 0 1]
 [0 0 1 1 0]
 [1 1 1 1 0]
 [0 0 1 1 1]
 [1 1 1 0 1]
 [1 0 1 1 0]
 [1 1 0 1 1]
 [1 1 1 0 0]
 [1 0 0 0 0]
 [1 1 1 1 1]
 [0 1 1 0 0]
 [1 1 1 0 0]
 [1 0 1 1 1]
 [1 1 1 0 0]
 [1 0 0 0 1]
 [0 1 0 0 0]
 [0 

In [29]:
# wing occupancy

print('before_inverting')
print(wing_occupancy_binary)
wing_occupancy_binary_inv = ~wing_occupancy_binary.astype(bool)
wing_occupancy_binary_inv = wing_occupancy_binary_inv.astype(int)
print('after_inverting')
print(wing_occupancy_binary_inv)

before_inverting
[1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 0]
after_inverting
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 1 1 1]


### Defining the room cost

In [30]:
print(room_capacity_metric_pr.shape)
print(room_energy_metric.shape)
print(room_lux_metric_pr.shape)
print(room_occupancy_inv.shape)

(50, 5)
(50, 5)
(50, 5)
(50, 5)


In [31]:
room_energy_metric

array([[ 91.,  78.,  95.,  93., 122.],
       [104.,  96., 111., 101.,  nan],
       [ 70.,  94., 110., 122., 110.],
       [119.,  72.,  99.,  nan,  nan],
       [ 95.,  98.,  74., 127.,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,

In [32]:
cost_of_rooms = (
    room_capacity_metric_pr
    *room_energy_metric
    *room_lux_metric_pr
    *room_occupancy_inv
    )
cost_of_rooms

array([[  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  nan],
       [  0.,   0., 110.,   0.,   0.],
       [  0.,   0.,   0.,  nan,  nan],
       [  0.,   0.,   0., 127.,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,

In [33]:

cost_of_wings = (

       wing_occupancy_binary_inv.reshape(NUM_WINGS,1)
       *  room_wing_cost_metric
)

cost_of_wings

array([[  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [303.69781494],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [304.44036865],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [300.51791382],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0.        ],
       [  0

In [34]:
cost_of_wings_pr = np.tile(cost_of_wings, (1, room_capacity_metric.shape[1]))
cost_of_wings_pr

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [303.69781494, 303.69781494, 303.69781494, 303.69781494,
        303.69781494],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ],
       [  0.        ,   0.        ,   0.   

In [35]:
final_cost_per_room = cost_of_rooms+cost_of_wings_pr
final_cost_per_room

array([[  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,  nan],
       [  0.,   0., 110.,   0.,   0.],
       [  0.,   0.,   0.,  nan,  nan],
       [  0.,   0.,   0., 127.,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,

In [36]:

flattened_array = final_cost_per_room.flatten()
non_zero_non_nan_indices = np.where(np.logical_and(flattened_array != 0, ~np.isnan(flattened_array)))

# Get the indices of the first 3 lowest values after excluding 0 and NaN
sorted_indices = np.argsort(flattened_array[non_zero_non_nan_indices])[:1]

# Convert the flattened indices to the original 2D indices
row_indices, col_indices = np.unravel_index(non_zero_non_nan_indices[0][sorted_indices], final_cost_per_room.shape)

# Get the values and positions
lowest_values = flattened_array[non_zero_non_nan_indices][sorted_indices]
positions = list(zip(row_indices, col_indices))

# Display the original array, flattened array, and result
print("Original Array:")
print(final_cost_per_room)


print("\nIndices of First 3 Lowest Values (excluding 0 and NaN):")
print(positions)

print("\nValues of First 3 Lowest Values (excluding 0 and NaN):", lowest_values)

Original Array:
[[  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  nan]
 [  0.   0. 110.   0.   0.]
 [  0.   0.   0.  nan  nan]
 [  0.   0.   0. 127.  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ n

In [37]:
positions[0][1]

2

In [38]:
best_wing =np.array( wing_df[['Wing_Number']].sort_values(by= ['Wing_Number']).reset_index(drop=True))[positions[0][0]][0]
best_wing

'C'

In [39]:
best_room = positions[0][1]
best_room

2